# 캐슬 디펜스

[baekjoon의 '캐슬 디펜스' 링크](https://www.acmicpc.net/problem/17135)

### 문제  
캐슬 디펜스는 성을 향해 몰려오는 적을 잡는 턴 방식의 게임이다. 게임이 진행되는 곳은 크기가 N×M인 격자판으로 나타낼 수 있다. 격자판은 1×1 크기의 칸으로 나누어져 있고, 각 칸에 포함된 적의 수는 최대 하나이다. 격자판의 N번행의 바로 아래(N+1번 행)의 모든 칸에는 성이 있다.  
  
성을 적에게서 지키기 위해 궁수 3명을 배치하려고 한다. 궁수는 성이 있는 칸에 배치할 수 있고, 하나의 칸에는 최대 1명의 궁수만 있을 수 있다. 각각의 턴마다 궁수는 적 하나를 공격할 수 있고, 모든 궁수는 동시에 공격한다. 궁수가 공격하는 적은 거리가 D이하인 적 중에서 가장 가까운 적이고, 그러한 적이 여럿일 경우에는 가장 왼쪽에 있는 적을 공격한다. 같은 적이 여러 궁수에게 공격당할 수 있다. 공격받은 적은 게임에서 제외된다. 궁수의 공격이 끝나면, 적이 이동한다. 적은 아래로 한 칸 이동하며, 성이 있는 칸으로 이동한 경우에는 게임에서 제외된다. 모든 적이 격자판에서 제외되면 게임이 끝난다.  
  
게임 설명에서 보다시피 궁수를 배치한 이후의 게임 진행은 정해져있다. 따라서, 이 게임은 궁수의 위치가 중요하다. 격자판의 상태가 주어졌을 때, 궁수의 공격으로 제거할 수 있는 적의 최대 수를 계산해보자.  
  
격자판의 두 위치 (r1, c1), (r2, c2)의 거리는 |r1-r2| + |c1-c2|이다.  

### 입력  
첫째 줄에 격자판 행의 수 N, 열의 수 M, 궁수의 공격 거리 제한 D가 주어진다. 둘째 줄부터 N개의 줄에는 격자판의 상태가 주어진다. 0은 빈 칸, 1은 적이 있는 칸이다.  
  
### 출력  
첫째 줄에 궁수의 공격으로 제거할 수 있는 적의 최대 수를 출력한다.  
  

In [2]:
from queue import PriorityQueue
from itertools import combinations

def targeting_enemy(N, M, D, board, std_y, std_x): # 가까운 적을 타겟팅하는 함수
    checked = [[D+1 for _ in range(M)] for _ in range(std_y)]
    # 적이 있는 공간에 최소거리 저장
    q = PriorityQueue() # 우선순위 큐
    q.put((1, std_x, std_y-1)) # 거리, x, y
    checked[std_y-1][std_x] = 1 # 성 바로 위에 있는 위치와의 거리는 1
    while not q.empty(): # 큐가 빌 때까지 루프
        dist, x, y = q.get() # 거리, x, y
        if board[y][x] == 1: # 만약 해당하는 좌표에 적이 있다면
            return (y, x) # 좌표 반환
        if 0<=(x-1)<M and abs(std_y-y)+abs(std_x-(x-1)) <= D: # 현재 좌표의 왼쪽 탐색
            if checked[y][x-1] < dist+1: # 만약 이미 더 적은 거리로 탐색되었다면 패스
                pass
            else: # 아니라면 해당 좌표를 큐에 넣고, 거리를 저장한다.
                q.put((dist+1, x-1, y))
                checked[y][x-1] = dist+1
        if 0<=(y-1) and abs(std_y-(y-1))+abs(std_x-x) <= D: # 현재 좌표의 위쪽 탐색
            if checked[y-1][x] < dist+1:
                pass
            else:
                q.put((dist+1, x, y-1))
                checked[y-1][x] = dist+1
        if 0<=(x+1)<M and abs(std_y-y)+abs(std_x-(x+1)) <= D: # 현재 좌표의 오른쪽 탐색
            if checked[y][x+1] < dist+1:
                pass
            else:
                q.put((dist+1, x+1, y))
                checked[y][x+1] = dist+1
    return None # 만약 적이 없다면 None 반환
    
def simulation(N, M, D, board, location, idx=0): # location의 위치에 궁수를 배치했을 때 잡을 수 있는 최대 적
    if idx == N: # 모든 적에 대해서 탐색 했을 때
        return 0 # 0 반환
    answer = 0 # 답
    point = 0 # 적을 맞춘 점수
    enemy_locs = set() # 맞춘 적들의 위치
    for l in location: # l의 위치에 궁수가 있을 때
        enemy_loc = targeting_enemy(N, M, D, board, N-idx, l)
        # 궁수와의 거리가 D보다 짧다는 조건하에 짧은 거리순, 좌우순으로 가장 빠른 적의 위치
        if enemy_loc: # 만약 만족하는 적이 있다면
            enemy_locs.add(enemy_loc) # 해당 좌표 저장
    point += len(enemy_locs) # 그리고 궁수가 맞춘 적의 수를 점수에 더한다.
    for y, x in enemy_locs:
        board[y][x] = 0 # 적이 궁수에 맞았음을 표시
    answer = point+simulation(N, M, D, board, location, idx+1)
    # 재귀적으로 진행
    for y, x in enemy_locs:
        board[y][x] = 1
    return answer # 답 반환

T = int(input("testcase num: ")) # 테스트 케이스 개수
print()
while T > 0:
    print("Input")
    answer = 0 # 답
    N, M, D = tuple(int(x) for x in input("N, M, D: ").split()) # 행, 열, 궁수 사거리
    board = [] # 보드
    for _ in range(N):
        row = [int(x) for x in input().split()]
        board.append(row)
        # 적의 배치 저장
    locations = list(combinations(list(range(M)), 3))
    # 궁수의 위치 경우의 수
    for location in locations:
        answer = max(answer, simulation(N, M, D, board, location))
        # 모든 경우의 수에 대해서 최대 포인트를 구한다.
    print("\nOutput")
    print(answer) # 답 출력
    print()
    T -= 1

testcase num: 6

Input
N, M, D: 5 5 1
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
1 1 1 1 1

Output
3

Input
N, M, D: 5 5 1
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
1 1 1 1 1
0 0 0 0 0

Output
3

Input
N, M, D: 5 5 2
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
1 1 1 1 1
0 0 0 0 0

Output
5

Input
N, M, D: 5 5 5
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1

Output
15

Input
N, M, D: 6 5 1
1 0 1 0 1
0 1 0 1 0
1 1 0 0 0
0 0 0 1 1
1 1 0 1 1
0 0 1 0 0

Output
9

Input
N, M, D: 6 5 2
1 0 1 0 1
0 1 0 1 0
1 1 0 0 0
0 0 0 1 1
1 1 0 1 1
0 0 1 0 0

Output
14

